# <center>PROJECT 1: COLOR COMPRESSION</center>

### <center> Class: 20CLC11</center>
### <center> Student: Mai Quý Trung - 20127370</center>

### <b> <u>Bước 1 </u>: Import thư viện </b>
Em sử dụng các thư viện như sau: 
- `matplotlib`: dùng `plt` để xuất ảnh dưới dạng figure và lưu lại dưới dạng png/pdf, dùng `img` để import ảnh và chuyển ảnh thành ma trận các pixel màu
- `numpy`: thư viện dùng để xử lí các thông tin trên các ma trận n chiều và random choice
- `scipy`: dùng cdist để tính khoảng cách gần nhất từ các điểm ảnh với các centroid sử dụng thuật toán euclidean

In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as img
import numpy as np
from scipy.spatial.distance import cdist
import random

### <b> <u>Bước 2 </u>: Viết thuật toán K Means (K-means clustering algorithm) </b>

Các bước thực hiện chi tiết thuật toán:
1.  Các thông số truyền vào:
- `img_1d`: ma trận 1 chiều các pixel màu của ảnh
- `k_clusters`: số lượng màu mong muốn (số lượng cluster để phân nhóm)
- `max_iter`: giá trị tối đa số lần lặp lại các bước phân nhóm
- `init_centroids`: chọn centroid theo 1 trong 2 kiểu `random` hoặc `in-pixels` (default là `random`)
2. Giá trị trả về: Hàm sẽ trả về các giá trị sau đây
- centroid: mảng chứa số lượng màu tương ứng với số nhóm được phân chia (cluster)
- labels: mảng ma trận các pixel màu với mỗi pixel được dán nhãn nhóm mà pixel đó thuộc về
3. Chi tiết hàm:
- Đầu tiên, tạo biến centroids là mảng trống, với centroid sẽ là các pixel màu đại diện cho từng group (cluster)
- Tiếp theo, nếu `init_centroids` là `random`: tạo biến check = False, rồi sẽ chọn vị trí bắt đầu (k_index) và chọn ra `k_clusters` màu cho mảng centroids. Sau đó centroids sẽ được gán các pixel màu đó từ `img_1d`
- Nếu `init_centroids` là `in-pixels`: sẽ lấy `k_clusters` màu bằng cách random từng màu R, G và B cho mỗi pixel, nếu ảnh là màu rgb thì centroids sẽ được random 3 pixel màu, nếu là rgba thì 4 màu
- Sau khi có được mảng centroids, ta sẽ tính khoảng cách từ mỗi pixel trong `img_1d` đến các centroids sử dụng hàm `cdist` với thuật toán `euclidean`
- Với khoảng cách đến các centroids của từng pixel màu, ta sẽ thực hiện việc nhán dãn (label) từng pixel màu với centroid gần với pixel đó nhất bằng hàm `argmin` của thư viện `numpy`
- Sau khi label các pixel màu, ta sẽ cho chạy vòng lặp với số lần lặp là `max_iter`, gán centroids lại bằng mảng rỗng, với số lượng centroid và vị trí của chúng ban đầu, ta sẽ thiết lập vị trí mới của chúng để có thể tập trung các pixel màu chính xác hơn bằng hàm `mean` tính trung bình của các pixel xung quanh cluster tương ứng
- Các vị trí centroid sau đó được update và ta sẽ thực hiện lại công việc tính khoảng cách đến các centroid và label từng pixel màu, cho đến hết số lần `max_iter`
- <b><u>Lưu ý</u>:</b> <i>`max_iter` càng lớn, độ chính xác phân vùng các nhóm màu càng cao nhưng sẽ đi đôi với việc hàm sẽ chạy lâu hơn</i>


In [2]:
def k_means_clustering_algorithm(img_1d, k_clusters, max_iter, init_centroids='random'):
    centroids, length = [], len(img_1d)
    if init_centroids == 'in-pixels':
        # step 1: randomly choose a k index
        check = False
        k_index = np.random.choice(length, k_clusters, check)
        #print(k_index)
        # initialize centroid from k_index
        centroids = img_1d[k_index, :]
        #print(centroids)
    elif init_centroids == 'random':
        # step 1: choose k_clusters colors and append them to centroid
        centroids = np.random.randint(255, size = (k_clusters, img_1d.shape[1]))
        #print(centroids)

    # step 2: find the distance from pixels to centroids using euclidean
    centroid_dist = cdist(img_1d, centroids, 'euclidean')

    # step 3: label pixels to their closest centroid cluster by measuring the minimum distance
    labels = np.array([np.argmin(i) for i in centroid_dist])

    # step 4: for loop to repeat the process max_iter times
    while max_iter > 0:
        temp = centroids
        centroids = []
        # step 5: for loop number of new centroids by creating new center based on the previous ones
        for index in range(0, k_clusters):
            # update centroids by taking mean
            if True:
                new_center = img_1d[index == labels].mean(axis=0)
                centroids.append(new_center)

        if init_centroids == 'in-pixels':
            for new in centroids:
                for old in temp:
                    if all(new == old): max_iter = 1

        # step 6: update centroids and calculate the centroid distance and labels again 
        centroids = np.vstack(centroids)
        centroid_dist, labels = cdist(img_1d, centroids, 'euclidean'), np.array([np.argmin(i) for i in centroid_dist])
        max_iter -= 1

    return centroids, labels

### <b> <u>Bước 3 </u>: Khởi tạo các giá trị ban đầu </b>
Hàm dưới đây sẽ hoạt động như sau:
- Cho người dùng nhập vào tên file
- Cho người dùng nhập vào định dạng file muốn xuất (png/pdf)
- Sau đó hàm sẽ xử lí chuỗi lấy tên của file (không chưa extension) và định dạng file muốn xuất

In [3]:
def initialize():
    filename = str(input("Enter filename: "))
    index = filename.find(".")
    name = filename[0:index]
    type = str(input("Enter format file (png/pdf): ")).lower()
    return filename, name, type

### <b> <u>Bước 4 </u>: Xử lí ảnh, chuyển đổi từ file raw sang ma trận các pixel màu </b>
Các bước hàm thực hiện như sau:
- Hàm nhận input argument là filename từ người dùng nhập vào
- Sau đó sử dụng thư viện `matplotlib.image` hàm `imread` để chuyển ảnh sang ma trận 3 chiều: chiều dài, chiều rộng và mỗi điểm ảnh là 1 ma trận gồm 3 hoặc 4 phần tử (3 cho màu `rgb` và 4 cho màu `rgba`)
- Chuyển đổi từ ma trận 2 chiều với các mảng màu rgb/rgba sang ma trận 1 chiều với hàm `reshape`
- Sau đó lưu các thông số chiều cao, chiều rộng và số lượng màu của ảnh vào và trả về các giá trị đó

In [4]:
def handling_picture(filename):
    image = img.imread(filename)
    h, w, c = image.shape[0], image.shape[1], image.shape[2]
    image = image.reshape(h * w, c)
    return image, h, w, c

### <b> <u>Bước 5 </u>: Hàm main, sử dụng K Means với centroid ban đầu là random </b>
Các bước thực hiện:
- Lấy filename và định dạng file muốn xuất từ hàm `initialize` được nêu trên
- Sau đó khởi tạo mảng với số lượng cluster theo thứ tự giảm dần: 7, 5, 3
- Chạy vòng lặp for trong mảng cluster
- Lấy ma trận 3 chiều các pixel màu, chiều cao, chiều rộng và loại màu từ hàm `handling_picture`
- Sau đó dùng thuật toán `k_means_clustering_algorithm` với số lượng cluster tương ứng, `max_iter = 30` và `init_centroid = 'random'`
- Hàm trả về centroids và labels
- Sau đó gán lại các giá trị màu từ centroids và labels vào ma trận ảnh
- Lúc này ma trận ảnh đang là ma trận 1 chiều, ta thực hiện hàm `reshape` để đưa về ma trận 2 chiều ban đầu
- Ảnh sau đó được thư viện `plt` thực hiện các hàm `imshow` và `show` để xuất ảnh ra figure và hàm `imsave` để lưu figure với định dạng tương ứng


In [5]:
def mainRandom():
    filename, name, type = initialize()
    k = [7, 5, 3]
    for idx in range(len(k)):
        image, h, w, c = handling_picture(filename)
        print("cluster =", k[idx])
        centroids, labels = k_means_clustering_algorithm(image, k[idx], 30, 'random')
        for i in range(0, image.shape[0]):
            image[i] = centroids[labels[i]]
        data = image.reshape(h, w, c)
        plt.imshow(data)
        plt.show()
        plt.imsave(name + "-cluster=" + str(k[idx]) + "." + type, data)


### <b> <u>Bước 5 </u>: Hàm main, sử dụng K Means với centroid ban đầu là in-pixels </b>
Hàm được thực hiện tương tượng như hàm `mainRandom` nhưng sử dụng thuật toán `k_means_clustering_algorithm` với `init_centroid = 'in-pixels'`

In [6]:
def mainInPixels():
    filename, name, type = initialize()
    k = [7, 5, 3]
    for idx in range(len(k)):
        image, h, w, c = handling_picture(filename)
        print("cluster =", k[idx])
        centroids, labels = k_means_clustering_algorithm(image, k[idx], 30, 'in-pixels')
        for i in range(0, image.shape[0]):
            image[i] = centroids[labels[i]]
        data = image.reshape(h, w, c)
        plt.imshow(data)
        plt.show()
        plt.imsave(name + "-cluster=" + str(k[idx]) + "." + type, data)


### <b> <u>Bước 5 </u>: Gọi hàm main và thực thi</b>

#### Hàm main dùng k means với centroid random

In [ ]:
#mainRandom()

#### Hàm main dùng k means với centroid in-pixels

In [8]:
mainInPixels()